In [ ]:
import matplotlib.pyplot as plt
import random
import numpy as np

# Parent Class

In [ ]:
class cache:
    def __init__(self, s):
        self.maxsize = s
        self.current_size = 0
        self.cache = []
        self.status = []
        self.hash = set()
    def cacheStatus(self):
        return self.status
    def printCache(self):
        return self.cache

## Random

In [ ]:
class RandomCache(cache):
    def __init__(self, s):
        cache.__init__(self,s)
    def updateCache(self, elem):
        if elem not in self.hash:
            self.status += ['miss']
            if self.current_size == self.maxsize:
                rand_num = random.randint(0,self.maxsize-1)
                self.cache.append(elem)
                self.hash.add(elem)
                del_elem = self.cache.pop(rand_num)
                self.hash.remove(del_elem)
            elif self.current_size < self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                self.current_size += 1
        else:
            self.status += ['hit']

In [ ]:
ref_array = [0, 1, 2, 0, 1, 3, 0, 3, 1, 2, 1]

In [ ]:
randC = RandomCache(3)

In [ ]:
for i,k in enumerate(ref_array):
    randC.updateCache(k)
    status = randC.cacheStatus()[i]
    print(status, randC.printCache())

## FIFO

In [ ]:
class FIFOCache(cache):
    def __init__(self, s):
        cache.__init__(self,s)
    def updateCache(self, elem):
        if elem not in self.hash:
            self.status += ['miss']
            if self.current_size == self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                del_elem = self.cache.pop(0)
                self.hash.remove(del_elem)
            elif self.current_size < self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                self.current_size += 1
        else:
            self.status += ['hit']

In [ ]:
ref_array = [0, 1, 2, 0, 1, 3, 0, 3, 1, 2, 1]

In [ ]:
fifo = FIFOCache(3)

In [ ]:
for i,k in enumerate(ref_array):
    fifo.updateCache(k)
    status = fifo.cacheStatus()[i]
    print(status, fifo.printCache())

## LFU

In [ ]:
class LFUCache(cache):
    def __init__(self, s):
        cache.__init__(self,s)
        self.hash = {}
        
    def extractMinIndex(self):
        minkey = min(self.hash, key=self.hash.get)
        return self.cache.index(minkey)
    
    def updateCache(self, elem):
        if elem not in self.hash:
            self.status += ['miss']
            if self.current_size == self.maxsize:
                del_elem = self.cache.pop(self.extractMinIndex())
                del self.hash[del_elem]
                self.cache.append(elem)
                self.hash[elem] = 1
            elif self.current_size < self.maxsize:
                self.cache.append(elem)
                self.hash[elem] = 1
                self.current_size += 1
        else:
            self.status += ['hit']
            self.hash[elem] += 1

In [ ]:
ref_array = [0, 1, 2, 0, 1, 3, 0, 3, 1, 2, 1]

In [ ]:
lfu = LFUCache(3)

In [ ]:
for i,k in enumerate(ref_array):
    lfu.updateCache(k)
    status = lfu.cacheStatus()[i]
    print(status, lfu.printCache())

## LRU

In [ ]:
class LRUCache(cache):
    def __init__(self, s):
        cache.__init__(self,s)
    def updateCache(self, elem):
        if elem not in self.hash:
            self.status += ['miss']
            if self.current_size == self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                del_elem = self.cache.pop(0)
                self.hash.remove(del_elem)
            elif self.current_size < self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                self.current_size += 1
        else:
            self.status += ['hit']
            del_elem = self.cache.pop(self.cache.index(elem))
            self.cache.append(del_elem)

In [ ]:
ref_array = [0, 1, 2, 0, 1, 3, 0, 3, 1, 2, 1]

In [ ]:
lru = LRUCache(3)

In [ ]:
for i,k in enumerate(ref_array):
    lru.updateCache(k)
    status = lru.cacheStatus()[i]
    print(status, lru.printCache())

## Oracle

In [ ]:
class OracleCache(cache):
    def __init__(self, s, load):
        cache.__init__(self,s)
        self.present = 0
        self.load = load
    
    def farthest(self):
        if len(self.load[self.present:])==1:
            return 0
        else:
            self.m = -500
            for c in self.cache:
                if c in self.load[self.present:]:
                    self.m = max(self.load[self.present:].index(c), self.m)
                else:
                    return self.cache.index(c)
            return self.cache.index(self.load[self.present:][self.m])
            
            
    def updateCache(self, elem):
        if elem not in self.hash:
            self.present += 1
            self.status += ['miss']
            if self.current_size == self.maxsize:
                del_elem = self.cache.pop(self.farthest())
                self.hash.remove(del_elem)
                self.cache.append(elem)
                self.hash.add(elem)
            
            elif self.current_size < self.maxsize:
                self.cache.append(elem)
                self.hash.add(elem)
                self.current_size += 1
        else:
            self.present += 1
            self.status += ['hit']

In [ ]:
orac = OracleCache(3, ref_array)

In [ ]:
ref_array = [0, 1, 2, 0, 1, 3, 0, 3, 1, 2, 1]

In [ ]:
for i,k in enumerate(ref_array):
    orac.updateCache(k)
    status = orac.cacheStatus()[i]
    print(status, orac.printCache())

## Plotting

### Workloads

In [ ]:
W_loopseq = []
for i in range(200):
    W_loopseq += [i for i in range(1,51)]

In [ ]:
W_noloc = list(np.random.randint(1,101, size=(10000,)))

In [ ]:
w_80ref = np.random.randint(1, 21, size=(8000,))
w_20ref = np.random.randint(21, 101, size=(2000,))
W_8020 = np.append(w_20ref, w_80ref, axis=0)
np.random.shuffle(W_8020)

In [ ]:
W = {'80-20': W_8020, 'no-locality': W_noloc, 'looping-seq': W_loopseq}

#### Each of the below functions take a bit time(2-3 min)

In [ ]:
def plot_noloc(workloads=W):
    A = workloads['no-locality']
    caches = [RandomCache, FIFOCache, LRUCache, LFUCache, OracleCache]
    graphs = ['k-', 'b|', 'g.', 'co', 'r-']
    Y_s, X_s = [], [i for i in range(1,101)]
    for n,ca in enumerate(caches):
        Y = []
        X = [i for i in range(1,101)]
        for i in range(1,101): #cache size
            if n==4:
                c = ca(i, list(A))
            else:
                c = ca(i)
            for j in A:
                c.updateCache(j)
            Y.append((c.cacheStatus().count('hit')/len(c.cacheStatus()))*100)
        Y_s.append(Y)
        
    plt.plot(X_s, Y_s[0], graphs[0],label='Random')
    plt.plot( X_s, Y_s[1], graphs[1], label='FIFO')
    plt.plot(X_s, Y_s[2], graphs[2], label='LRU')
    plt.plot(X_s, Y_s[3], graphs[3], label='LFU')
    plt.plot(X_s, Y_s[4], graphs[4], label='Oracle')
    plt.legend()
    plt.show()

In [ ]:
%%time
plot_noloc()

In [ ]:
def plot_8020(workloads=W):
    A = workloads['80-20']
    caches = [RandomCache, FIFOCache, LRUCache, LFUCache, OracleCache]
    graphs = ['k-', 'b|', 'g.', 'co', 'r-']
    Y_s, X_s = [], [i for i in range(1,101)]
    for n,ca in enumerate(caches):
        Y = []
        X = [i for i in range(1,101)]
        for i in range(1,101): #cache size
            if n==4:
                c = ca(i, list(A))
            else:
                c = ca(i)
            for j in A:
                c.updateCache(j)
            Y.append((c.cacheStatus().count('hit')/len(c.cacheStatus()))*100)
        Y_s.append(Y)
        
    plt.plot(X_s, Y_s[0], graphs[0],label='Random')
    plt.plot( X_s, Y_s[1], graphs[1], label='FIFO')
    plt.plot(X_s, Y_s[2], graphs[2], label='LRU')
    plt.plot(X_s, Y_s[3], graphs[3], label='LFU')
    plt.plot(X_s, Y_s[4], graphs[4], label='Oracle')
    plt.legend()
    plt.show()

In [ ]:
%%time
plot_8020()

In [ ]:
def plot_loopseq(workloads=W):
    A = workloads['looping-seq']
    caches = [RandomCache, FIFOCache, LRUCache, LFUCache, OracleCache]
    graphs = ['k-', 'b|', 'g.', 'co', 'r-']
    Y_s, X_s = [], [i for i in range(1,101)]
    for n,ca in enumerate(caches):
        Y = []
        X = [i for i in range(1,101)]
        for i in range(1,101): #cache size
            if n==4:
                c = ca(i, list(A))
            else:
                c = ca(i)
            for j in A:
                c.updateCache(j)
            Y.append((c.cacheStatus().count('hit')/len(c.cacheStatus()))*100)
        Y_s.append(Y)
        
    plt.plot(X_s, Y_s[0], graphs[0],label='Random')
    plt.plot( X_s, Y_s[1], graphs[1], label='FIFO')
    plt.plot(X_s, Y_s[2], graphs[2], label='LRU')
    plt.plot(X_s, Y_s[3], graphs[3], label='LFU')
    plt.plot(X_s, Y_s[4], graphs[4], label='Oracle')
    plt.legend()
    plt.show()

In [ ]:
%%time
plot_loopseq()